In [1]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

Mounted at /content/gdrive


In [2]:
import pandas as pd
import numpy as np
import nltk
import regex
import gensim
from gensim.utils import lemmatize
from gensim.utils import simple_preprocess
import spacy #python3 -m spacy download en
from spacy.lang.en.stop_words import STOP_WORDS as en
from wordcloud import WordCloud
from matplotlib import pyplot as plt

In [3]:
df = pd.read_csv('/content/gdrive/My Drive/Research/Datasets/ft2020all.csv')
# df[df['Date'] == 'yesterday']['Date']
df.loc[df['Date'] == 'yesterday', 'Date'] = '2020-07-14'
df.loc[df['Date'] == '5 hours ago', 'Date'] = '2020-07-14'
df.loc[df['Date'] == '2 hours ago', 'Date'] = '2020-07-14'
df.loc[df['Date'] == '4 hours ago', 'Date'] = '2020-07-14'
df['Date'] =pd.to_datetime(df.Date)
df =df.dropna().set_index('Date').sort_index()
texts = df.Article.values.tolist()
df = df.assign(weekoftheyear=pd.PeriodIndex(df.index, freq='D'))
ind = (df.weekoftheyear.unique())
len(df)
df

,Unnamed: 0,Title,Article,weekoftheyear
Date,,,,
2020-01-01,0,"Hello, Robot: machines designed for comfort — ...",In the 1930s there were two kinds of robot sto...,2020-01-01
2020-01-01,24,ADHD app: game theory,"For parents with young children, video games c...",2020-01-01
2020-01-01,25,The EU needs to learn the language of power,More than three years have passed since EU lea...,2020-01-01
2020-01-01,26,Harry Potter at 40: would he be a financial wi...,"Fans of Harry Potter, immortalised in fiction ...",2020-01-01
2020-01-01,27,South Korean export decline slows on China shi...,South Korean exports marked the smallest decli...,2020-01-01
...,...,...,...,...
2020-07-14,21036,India takes a risk by encouraging national int...,Soon after India announced it was banning 59 p...,2020-07-14
2020-07-14,21034,UK’s largest accounting firms lambasted by wat...,The UK’s biggest accounting firms have been cr...,2020-07-14
2020-07-14,21033,Petropavlovsk revival at risk after boardroom ...,Petropavlovsk shareholders thought the company...,2020-07-14


In [4]:
#regular expression: formatting text
def formatting(texts):
    texts = [regex.sub('\s+', ' ', text) for text in texts] #remove new lines
    texts = [regex.sub("\"", "", text) for text in texts] #remove quotations
    texts = [regex.sub("\'", "", text) for text in texts] #remove quotations
    return texts
formatted_text = formatting(texts)

In [5]:
stop_words = list(en)
stop_words.extend(['per','bn','mr','cent','say','new','use','per','not', 'would', 'say', 'could', '_', 'be', 'know', 'good', 'go', 'get', 'do', 'done', 'try', 'many', 'some', 'nice', 'thank', 'think', 'see', 'rather', 'easy', 'easily', 'lot', 'lack', 'make', 'want', 'seem', 'run', 'need', 'even', 'right', 'line', 'even', 'also', 'may', 'take', 'come','q','w','e','r','t','y','u','i','o','p','a','s','d','f','g','h','j','k','l','z','x','c','v','b','n','m','ty'])
lemma = spacy.load('en', disable=['parser', 'ner'])

def token(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence),deacc=True)) 

def bigram_trigram(texts):
    bigram = gensim.models.Phrases(texts)
    trigram = gensim.models.Phrases(bigram[texts])
    bigram_ = gensim.models.phrases.Phraser(bigram)
    trigram_ = gensim.models.phrases.Phraser(trigram)
    texts = [[word for word in doc if word not in stop_words] for doc in texts] 
    bi = [bigram_[text] for text in texts]
    tri = [trigram_[bigram_[b]] for b in bi]
    return tri
def processing(texts, stop_words=stop_words):
    texts = [[word for word in doc if word not in stop_words] for doc in texts]
    output = []
    for text in texts:
        doc = lemma(" ".join(text)) 
        output.append([token.lemma_ for token in doc])
    # remove stopwords once more after lemmatization
    output = [[word for word in doc if word not in stop_words] for doc in output]
    return output

tokens = list(token(formatted_text))
bi_tri = bigram_trigram(tokens)
final_data = processing(bi_tri)
df['ProcessedArticles'] = final_data

/usr/local/lib/python3.6/dist-packages/gensim/models/phrases.py:598: UserWarning: For a faster implementation, use the gensim.models.phrases.Phraser class
  warnings.warn("For a faster implementation, use the gensim.models.phrases.Phraser class")


In [6]:
def search_risk(text,n):
    word = r'\W*([\w]+)'
    risk = regex.findall(r'{}\W*{}{}'.format(word*n,'risk',word*n), text)
    uncertainty = regex.findall(r'{}\W*{}{}'.format(word*n,'uncertainty',word*n), text)
    uncertain = regex.findall(r'{}\W*{}{}'.format(word*n,'uncertain',word*n), text)
    fear = regex.findall(r'{}\W*{}{}'.format(word*n,'fear',word*n), text)
    return risk+uncertainty+uncertain+fear

In [7]:
risk = [(search_risk(' '.join(data),10)) for data in final_data ]

In [ ]:
# df['Risk'] = risk
# len(risk)
# df.to_csv('/content/gdrive/My Drive/Research/Datasets/final_risk_10ENG.csv')
# df = pd.read_csv('/content/gdrive/My Drive/Research/Datasets/final_risk_10ENG.csv').set_index('Date')
# risk = df.Risk.tolist()

In [ ]:
stop_words.extend(['latest_storie','january','february','march','april','may','june','july','august','september','october','november','december','y_asset','likely','window_share','calere'])
df = df.assign(weekoftheyear=pd.PeriodIndex(df.index, freq='W'))
def frequency(data,top_N,term):
  freq = []
  for i in df['weekoftheyear'].unique():
    week_data = df[(df['weekoftheyear'] == i)][term]
    flat_list = [item for sublist in week_data for item in sublist]
    flat_list1 = [item for sublist in flat_list for item in sublist]
    output = [word for word in flat_list1 if word not in stop_words]
    word_dist = nltk.FreqDist(output)
    temp = pd.DataFrame(word_dist.most_common(),
                      columns=['Word', 'Frequency'])
    temp['Percentage'] = temp.Frequency / len(temp.Word) * 100
    freq.append(temp.head(20))
  return freq

NameError: ignored

In [ ]:
risk = frequency(df,20,'Risk')

In [ ]:
#  for i in range(len(risk)): 
#   try:
#     word_dict = dict(zip(risk[i].Word, risk[i].Frequency))

#     wordcloud = WordCloud(background_color="white", width=900,height=500, max_words=1628,relative_scaling=1,normalize_plurals=False).generate_from_frequencies(word_dict)
#     plt.imshow(wordcloud)
#     print(('WEEK===' + str(i+1)))
#     plt.axis("off")
#     plt.show()
#   except:
#     print('no data')

In [ ]:
df_risk = []
# df = df.assign(weekoftheyear=pd.PeriodIndex(df.index, freq='M'))
for i in range(len(risk)):
  a = (risk[i].Word.head(5).to_list())
  df_risk.append(a)
df_risk = pd.DataFrame(df_risk,columns=['Word1','Word2','Word3','Word4','Word5']).set_index(df.groupby('weekoftheyear').sum().index)
print(df_risk.to_latex())

In [ ]:
df.groupby('weekoftheyear').sum()

In [ ]:
# [week.weekofyear for week in df.index]
i=2
for week in df.index:
  if week.weekofyear == i:
    print(week)
    i=i+1